# MOSViz example notebook

Note: We silence most warnings for now. For debugging, you can comment out the next cell and then restart the kernel to re-enable warnings.

In [ ]:
import warnings
warnings.simplefilter('ignore')

This starts MOSViz.

In [ ]:
from jdaviz.configs.mosviz.helper import MosViz as Mosviz

mosviz = Mosviz()
mosviz.app

But before we can use it, we need some data.

The MOSViz parsers accept lists of `Spectrum1D`, `SpectralCube`, and `CCDData` for 1D, 2D, and image data, respectively. Alternatively, users can also provide lists of file paths and MOSViz will internally attempt to parse them as their respective data types.

In [ ]:
# Run this cell if your desired data path is a temporary directory.

import tempfile
data_dir = tempfile.gettempdir()

In [ ]:
# *** OR ***
# Enable and run this cell if you have a different data path.
# Replace the value with your real path.
import glob

data_dir = '/Users/rosteen/Data/NIRSPEC/mossim_prism/'

# Create the lists of data products
slit_id = []
spectra_1d = []
spectra_2d = []
cutouts = []
for file_path in glob.glob(str(data_dir+'/*')):
    if ('x1d' in file_path):
        spectra_1d.append(file_path)
        slitid = file_path[74:79]
        slit_id.append(slitid)
    elif ('s2d' in file_path):
        spectra_2d.append(file_path)
    elif ('cutout' in file_path):
        cutouts.append(file_path)     

#Sort the slit ids and filenames
slit_id_sorted = sorted (slit_id)    
spectra_1d_sorted = sorted(spectra_1d)
spectra_2d_sorted = sorted(spectra_2d)
cutouts_sorted = sorted(cutouts)

#mosviz.load_data(spectra_1d_sorted[0:10], spectra_2d_sorted[0:10], images=cutouts_sorted[0:10])
mosviz.load_data(spectra_1d_sorted, spectra_2d_sorted, images=cutouts_sorted)

In [ ]:
print(f'My data is in {data_dir}')

Now that we have the data files extracted into `data_dir` we can load our files from that path.

In [ ]:
mosviz.load_data(directory=data_dir, instrument="nirspec")

WARNING: If the images did not load, you need to make sure that the number of images matches the number of spectra 1d and spectra 2d, as well as ensuring the location of the images is in a subdirectory of `data_dir` called `images`, `cutouts`, or `mosviz_cutouts`.

### Alternate ways to load data into Mosviz

Alternatively, we can run the following 2 to 3 cells to extract data from box and generate it into lists that can be loaded separately.

Once we have the lists populated, we can load them in the app.

Once loaded below, click on one of the rows in the table to have your data displayed.

If no images are provided, MOSViz can still display the spectra.

In [ ]:
mosviz_no_images = Mosviz()
mosviz_no_images.app

In [ ]:
mosviz_no_images.load_data(spectra_1d, spectra_2d)

Images can be added later.

In [ ]:
mosviz_no_images.load_images(images)
mosviz_no_images.load_metadata(images)

In [ ]:
mosviz.app.template

### Testing zoom-to-slit

In [ ]:
s2dv = mosviz.app.get_viewer('spectrum-2d-viewer')

In [ ]:
print(s2dv.axis_y.scale.max, s2dv.axis_y.scale.min)

In [ ]:
s2dv.layers[0].layer.meta

In [ ]:
from jdaviz.configs.mosviz.plugins.slit_overlay import jwst_header_to_skyregion

sky_region = jwst_header_to_skyregion(s2dv.layers[0].layer.meta)
sky_region

In [ ]:
wcs = s2dv.layers[0].layer.coords
wcs

In [ ]:
upper_y = wcs.celestial.pixel_to_world(0,0)
lower_y = wcs.celestial.pixel_to_world(25,0)
print(upper_y)
print(lower_y)

In [ ]:
iv = mosviz.app.get_viewer("image-viewer")

In [ ]:
iv.data()[0].meta

In [ ]:
iv.data()[0].coords

In [ ]:
s2dv.layers[0].layer.data.shape

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord

ra_center = 215.03883612*u.deg
dec_center = 52.89044753*u.deg
height=0.0007385744055694102*u.deg
width=

In [ ]:
im_wcs = iv.layers[0].layer.coords

upper = SkyCoord(ra_center+height, dec_center)
print(im_wcs.world_to_pixel(upper))

lower = SkyCoord(ra_center-height, dec_center)
print(im_wcs.world_to_pixel(lower))

In [ ]:
upper = SkyCoord(ra_center, dec_center+height)
print(im_wcs.world_to_pixel(upper))

lower = SkyCoord(ra_center, dec_center-height)
print(im_wcs.world_to_pixel(lower))

In [ ]:
upper_pixel = im_wcs.world_to_pixel(upper)
lower_pixel = im_wcs.world_to_pixel(lower)

In [ ]:
upper_pixel[1]-lower_pixel[1]